# Get Started in 10 Minutes

```{article-info}
:author: Altay Sansal
:date: "{sub-ref}`today`"
:read-time: "{sub-ref}`wordcount-minutes` min read"
:class-container: sd-p-0 sd-outline-muted sd-rounded-3 sd-font-weight-light
```

In this page we will be showing basic capabilities of MDIO.

For demonstration purposes, we will ingest the **remote** Teapot Dome open-source dataset.
The dataset details and licensing can be found at the [SEG Wiki](https://wiki.seg.org/wiki/Teapot_dome_3D_survey).

We are using the 3D seismic stack dataset named `filt_mig.sgy`.

The full HTTP link for the dataset (hosted on AWS): http://s3.amazonaws.com/teapot/filt_mig.sgy

```{warning}
For plotting and remote ingestion the notebook requires [Matplotlib](https://matplotlib.org/) and `aiohttp`
as a dependency. Please install it before executing using `pip install matplotlib aiohttp` or
 `conda install matplotlib aiohttp`.
```

## Defining the SEG-Y Dataset

Since MDIO 0.8 we can directly ingest remote SEG-Y files! The file is 386 MB in size. To make the header scan performant we can also
set up an environment variable for MDIO. See here for when to use this:
[Buffered Reads in Ingestion](https://mdio-python.readthedocs.io/en/stable/usage.html#buffered-reads-in-ingestion).

The dataset is irregularly shaped, however it is padded to a rectangle with zeros (dead traces). We will see that later at the live mask plotting.

The following environment variables are essential here:
- `MDIO__IMPORT__CLOUD_NATIVE` tells MDIO to do buffered reads for headers due to remote file.
- `MDIO__IMPORT__SAVE_SEGY_FILE_HEADER` makes MDIO save the SEG-Y specific file headers (text, binary) which is not strictly necessary for consumption and is disabled by default.

In [ ]:
import os

os.environ["MDIO__IMPORT__CLOUD_NATIVE"] = "true"
os.environ["MDIO__IMPORT__SAVE_SEGY_FILE_HEADER"] = "true"

input_url = "http://s3.amazonaws.com/teapot/filt_mig.sgy"

## Ingesting to MDIO

To do this, we can use the convenient SEG-Y to MDIO converter.

The `inline` and `crossline` values are located at bytes `181` and `185`. Note that this doesn't match any SEG-Y standards.

MDIO uses [TGSAI/segy][segy-github] to parse the SEG-Y; the field names conform to its canonical keys defined in [SEGY Binary Header Keys][segy-binary-keys] and [SEGY Trace Header Keys][segy-trace-keys]. Since MDIO v1 we also introduced templates for common seismic data types. For instance, we will be using the `PostStack3DTime` template [here][mdio-template-poststack3dtime], which expects the same canonical keys.

In summary, we will use the byte locations as defined for ingestion.

[segy-github]: https://github.com/TGSAI/segy
[segy-binary-keys]: https://github.com/TGSAI/segy/blob/main/src/segy/standards/fields/binary.py
[segy-trace-keys]: https://github.com/TGSAI/segy/blob/main/src/segy/standards/fields/trace.py
[mdio-template-poststack3dtime]: https://github.com/TGSAI/mdio-python/blob/main/src/mdio/builder/templates/seismic_3d_poststack.py

In [ ]:
import matplotlib.pyplot as plt
from segy.schema import HeaderField
from segy.standards import get_segy_standard

from mdio import segy_to_mdio
from mdio.builder.template_registry import get_template

teapot_trace_headers = [
    HeaderField(name="inline", byte=181, format="int32"),
    HeaderField(name="crossline", byte=185, format="int32"),
    HeaderField(name="cdp_x", byte=189, format="int32"),
    HeaderField(name="cdp_y", byte=193, format="int32"),
]

rev0_segy_spec = get_segy_standard(0)
teapot_segy_spec = rev0_segy_spec.customize(trace_header_fields=teapot_trace_headers)

mdio_template = get_template("PostStack3DTime")
segy_to_mdio(
    input_path=input_url,
    output_path="filt_mig.mdio",
    segy_spec=teapot_segy_spec,
    mdio_template=mdio_template,
    overwrite=True,
)

It only took a few seconds to ingest, since this is a very small file.

However, MDIO scales up to TB (that's ~1,000 GB) sized volumes!

## Opening the Ingested MDIO File

Let's open the MDIO file with the `open_mdio` function. This will return a pretty [`xarray`][xarray] representation with our standardized format.

[xarray]: https://docs.xarray.dev

In [ ]:
from mdio import open_mdio

dataset = open_mdio("filt_mig.mdio")
dataset

## Querying Metadata

Now let's look at the file text header saved in the `segy_file_header` metadata variable.

In [ ]:
print(dataset["segy_file_header"].attrs["textHeader"])

Since we saved the binary header, we can look at that as well.

In [ ]:
dataset["segy_file_header"].attrs["binaryHeader"]

## MDIO Grid, Dimensions, and Related Attributes

MDIO has an abstraction for an N-dimensional grid.
MDIO also has named dimensions, so we can see which dimension (axis) corresponds to which name.

In [ ]:
dataset.sizes

In [ ]:
dataset.inline

In [ ]:
dataset.crossline

## Fetching Data and Plotting

Now we will demonstrate getting an inline from MDIO.

Since MDIO v1 we are using Xarray under the hood, so we can use its convenient indexing. It also handles
the plotting with proper dimension coordinate labels.

MDIO stores summary statistics. We can calculate the standard deviation (std) value of the dataset to adjust the gain.

In [ ]:
from mdio.builder.schemas.v1.stats import SummaryStatistics

stats = SummaryStatistics.model_validate_json(dataset["amplitude"].attrs["statsV1"])
std = ((stats.sum_squares / stats.count) - (stats.sum / stats.count) ** 2) ** 0.5

il_dataset = dataset.sel(inline=278)
il_amp = il_dataset["amplitude"].T
il_amp.plot(vmin=-2 * std, vmax=2 * std, cmap="gray_r", yincrease=False);

Let's do the same with a time slice.

We will display two-way-time at 1,000 ms.

Note that since we parse the X/Y coordinates, we can plot time slice in real world coordinates.

In [ ]:
twt_data = dataset["amplitude"].sel(time=1000)
twt_data.plot(vmin=-2 * std, vmax=2 * std, cmap="gray_r", x="cdp_x", y="cdp_y");

We can also overlay live mask with the time slice. However, in this example, the dataset is zero-padded.

The live trace mask will always show True (yellow).

In [ ]:
trace_mask = dataset.trace_mask[:]

twt_data.plot(vmin=-2 * std, vmax=2 * std, cmap="gray_r", x="cdp_x", y="cdp_y", alpha=0.5, figsize=(8, 5))
trace_mask.plot(vmin=0, vmax=1, x="cdp_x", y="cdp_y", alpha=0.5);

## Query Headers

We can query headers for the whole dataset very quickly because they are separated from the seismic wavefield.

Let's get all the headers for X and Y coordinates in this dataset.

Note that the header maps will still share the geometry/grid of the dataset!

The `compute` property fetches the lazily opened MDIO values.

In [ ]:
dataset.headers["cdp_x"].compute()

In [ ]:
dataset.headers["cdp_y"].compute()

As we mentioned before, we can also get specific dataset slices of headers while fetching a slice.

Let's fetch a crossline; we are still using some previous parameters.

Since the sliced dataset contains the headers as well, we can plot the headers on top of the image.

Full headers can be mapped and plotted as well, but we won't demonstrate that here.

In [ ]:
xl_dataset = dataset.sel(crossline=100)  # slices everything available in MDIO dataset!

cdp_x_header = xl_dataset["cdp_x"]
cdp_y_header = xl_dataset["cdp_y"]
image = xl_dataset["amplitude"].T

# Build plot from here
gs_kw = {"height_ratios": (1, 8)}
fig, (hdr_ax, img_ax) = plt.subplots(2, 1, figsize=(6, 6), gridspec_kw=gs_kw, sharex="all")
hdr_ax2 = hdr_ax.twinx()

cdp_x_header.plot(ax=hdr_ax, c="red")
cdp_y_header.plot(ax=hdr_ax2, c="black")
image.plot(ax=img_ax, vmin=-2 * std, vmax=2 * std, cmap="gray_r", yincrease=False, add_colorbar=False)
img_ax.set_title("")
hdr_ax.set_xlabel("")

plt.tight_layout()

## MDIO to SEG-Y Conversion

Finally, let's demonstrate going back to SEG-Y.

We will use the convenient `mdio_to_segy` function and write it out as a round-trip file.

The output spec can be modified if we want to write things to different byte locations, etc, but we will use the same one as before.

In [ ]:
from mdio import mdio_to_segy

mdio_to_segy(
    input_path="filt_mig.mdio",
    output_path="filt_mig_roundtrip.sgy",
    segy_spec=teapot_segy_spec,
)

## Validate Round-Trip SEG-Y File

We can validate if the round-trip SEG-Y file is matching the original using [TGSAI/segy][segy-github].

Step by step:
* Open original file
* Open round-trip file
* Compare text headers
* Compare binary headers
* Compare 100 random headers and traces

[segy-github]: https://github.com/TGSAI/segy

In [ ]:
import numpy as np
from segy import SegyFile

original_segy = SegyFile(input_url)
roundtrip_segy = SegyFile("filt_mig_roundtrip.sgy")

# Compare text header
assert original_segy.text_header == roundtrip_segy.text_header

# Compare bin header
assert original_segy.binary_header == roundtrip_segy.binary_header

# Compare 100 random trace headers and traces
rng = np.random.default_rng()
rand_indices = rng.integers(low=0, high=original_segy.num_traces, size=100)
for idx in rand_indices:
    np.testing.assert_equal(original_segy.trace[idx], roundtrip_segy.trace[idx])

print("Files identical!")